# **Librerías**

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import json
import os

from google.colab import drive
from google.colab import files


In [2]:
drive.mount('/content/drive')

BASE = "/content/drive/MyDrive/NeuroBot/models"

MODEL_PATH  = f"{BASE}/best_model.pt"
CONFIG_PATH = f"{BASE}/config.json"
CLASSES_PATH = f"{BASE}/classes.json"

print("Modelo:", MODEL_PATH)
print("Config:", CONFIG_PATH)
print("Clases:", CLASSES_PATH)


Mounted at /content/drive
Modelo: /content/drive/MyDrive/NeuroBot/models/best_model.pt
Config: /content/drive/MyDrive/NeuroBot/models/config.json
Clases: /content/drive/MyDrive/NeuroBot/models/classes.json


In [4]:
# Reparar config.json si está incompleto
fixed_config = {
    "img_size": 224,
    "batch": 16,
    "num_classes": 3,
    "mean": [0.485, 0.456, 0.406],
    "std": [0.229, 0.224, 0.225],
    "classes": ["asfalto", "cesped", "grava"],
    "arch": "NeuroBotCNN",
    "saved": "fixed-manually"
}

with open(CONFIG_PATH, "w") as f:
    json.dump(fixed_config, f, indent=2)

print("✔️ config.json reparado correctamente")


✔️ config.json reparado correctamente


# **Cargar configuración y clases**

In [5]:
with open(CONFIG_PATH, "r") as f:
    config = json.load(f)

with open(CLASSES_PATH, "r") as f:
    class_names = json.load(f)

IMG_SIZE = config["img_size"]
mean = config["mean"]
std  = config["std"]
num_classes = config["num_classes"]

print("Clases:", class_names)
print("IMG_SIZE:", IMG_SIZE)


Clases: ['asfalto', 'cesped', 'grava']
IMG_SIZE: 224


# **Reconstruir la CNN**

In [6]:
class NeuroBotCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.fc(self.conv(x))


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = NeuroBotCNN(num_classes).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

print("\nModelo cargado y listo 🚀")



Modelo cargado y listo 🚀


# **Transformación para imágenes nuevas**

In [7]:
infer_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


# **Función para predecir una imagen**

In [8]:
def predecir_imagen(path):
    img = Image.open(path).convert("RGB")
    tensor = infer_tf(img).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        pred = model(tensor)
        _, idx = torch.max(pred, 1)
        clase = class_names[idx.item()]

    return img, clase


# **Subir imágenes reales desde tu PC / Celular**

In [14]:
uploaded = files.upload()

for fname in uploaded.keys():
    print("\nProcesando:", fname)
    img, pred = predecir_imagen(fname)

    plt.figure(figsize=(4,4))
    plt.imshow(img)
    plt.title("Predicción: " + pred.upper())
    plt.axis("off")
    plt.show()
